# Generadores y Corrutinas (Generators and Corroutines)

## Tabla de contenidos
- [Como crear generadores](#creacion)
<a href='#creacion'></a>

- [Generadores infinitos](#infinito)
<a href='#infinito'></a>

Los generadores son funciones que se comportan como iteradores generando los valores que retornan uno a uno.

Un generador ocupa `yield` para retornar un único valor cuando se le es solicitado. El efecto lateral es que los generadores pueden ser infinitamente largos porque se puede seguir solicitando por siempre.

Hay una variación en la sintaxís de los generadores que crean corrutinas. Las corrutinas son funciones que permiten el multitasking sin requerir múltiples threads o procesos. 

Mientras que los generadores pueden retornar values a quien se lo solicita basado en los argumentos iniciales, las corrutinas permiten dos formas de comunicación con la función que la llama mientras corre. La implementación de corrutinas en Python es a través del módulo `asyncio`.

Si las corrutinas funcionan para lo que usted necesita, pueden ofrecer una enorme mejora en el rendimiento.

## Generadores

A continuación se presentan las principales ventajas y desventajas de los generadores.

**Ventajas**

- Los generadores son usualmente más simples que escribir que una función que genera listas. Sólo se necesita el valor `yield`.
- Los ítems pueden ser procesados uno a la vez, así que generalmente no hay necesidad de mantener toda la lista en la memoria.
- Los resultados pueden depender de factores externos. Sólo se genera el valor cuando es solicitado.
- Los generadores son "flojos". Esto quiere decir, que si se usan solo los diez primeros resultados de un generador, el resto no va a ser calculado.

**Desventajas**
- Los resultados sólo están disponibles una vez. Después de procesar los resultados de un generador, no se pueden usar de nuevo.
- El tamaño es desconocido, incluso puede ser infinito.
- No es posible usar slices, así que mi_generador[3:10] no va a funcionar. Se puede usar itertools.islice para esto.
- No se puede indexar generadores, así que mi_generador[3] no funcionará.

<a id='creacion'></a>

## Como crear generadores

El generador más simple que se puede crear es una función que contiene una declaración `yield` en vez de `return`. La diferencia principal con funciones regulares que contienen un `return`, es que se pueden tener muchos `yield` en la función.

In [2]:
def mi_generador():
    yield 5
    yield "Buenos días"
    yield 25
    return "Hemos llegado al final"

resultado = mi_generador()

In [5]:
resultado

<generator object mi_generador at 0x000002A0240E7900>

In [6]:
len(resultado)

TypeError: object of type 'generator' has no len()

In [8]:
resultado[:]

TypeError: 'generator' object is not subscriptable

In [9]:
list(resultado)

[5, 'Buenos días', 25]

In [10]:
list(resultado)

[]

Como se puede observar, no se obtiene mucha información valiosa al con `repr()`, `len()` o slices. Además, utilizar `list()` una segunda oportunidad no funciona porque el generador ya está "cansado".

Además, puede que haya notado que el valor de `return` pareciera que ha desaparecido. No es el caso, el valor de return todavía se puede utilizar, pero como el valor para la excepción `StopIteration` producidad por el generador para indicar que está "exhausto".

In [11]:
def segundo_generador():
    yield "Algún valor"
    return "El fin del generador"

resultado = segundo_generador()

In [12]:
next(resultado)

'Algún valor'

In [13]:
next(resultado)

StopIteration: El fin del generador

Mencionar que nosotros podemos manejar la excepción, de `StopIteration`, de la siguiente manera (por ejemplo).

In [19]:
def flojo():
    print("Antes del yield statement")
    yield "He llegado a yield"
    print("Después del yield statement")
    
generador = flojo()

In [20]:
next(generador)

Antes del yield statement


'He llegado a yield'

In [21]:
try:
    next(generador)
except StopIteration:
    print("He llegado al final, pues se ha producido una excepción del tipo StopIteration")

Después del yield statement
He llegado al final, pues se ha producido una excepción del tipo StopIteration


In [22]:
for item in flojo():
    print(item)

Antes del yield statement
He llegado a yield
Después del yield statement


Para el correcto manejo de los generadores, siempre tiene que manejar usted la excepción `StopIteration`, o usar un loop u otra estructura que maneje `StopIteration` de forma implícita.

<a id='infinito'></a>

## Creando generadores infinitos

Crear un generador sin fin es sencillo. En vez de tener `yield` <valor> como en las funciones previas, ocupamos `yield` dentro de un loop infinito, así podemos hacer un generador infinito.

In [25]:
def contar(start=0, steps=1, stop= None):
    n = start
   
    while stop is not None and n < stop:
        yield n
        n += steps
        
list(contar(5, 1, 10))

[5, 6, 7, 8, 9]

**Debido a la naturaleza infinita de los generadores, se requiere precaución. Sin la variable stop, hacer list(contar()) puede resultar en un loop infinito que puede resultar en quedarse sin memoria de forma sumamente rápida.**

Lo anterior es en esencia, un for loop de toda la vida, la principal diferencia entre esto y el método regular de retornar una lista de items is que el `yield` statement retorna un item a la vez, lo que significa que solo se necesita calcular el ítem solicitado y no se deben de mantener todos los resultados en la memoria.